In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# this is the EM specific package for querying the EM data
from caveclient import CAVEclient


from meshparty import trimesh_io, trimesh_vtk
from meshparty import skeletonize, skeleton_io, skeleton
import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)
# # # if you have not yet setup this computer, uncomment this below line
# # # paste the token from the website in, and run the line


In [ ]:
# client.auth.save_token(token="fca0294ed408d104e0eebe74514c744b", overwrite=True)

# # # then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
client.materialize.get_versions()

In [ ]:
client.materialize.get_tables()

In [ ]:
client.materialize.get_table_metadata('nerve_bundle_fibers_v0')

# Create the pre_to_mn_df 
with index (segID,has_soma,sensory,neck,local) 
x 
(side, nerve,segment,function,muscle,rank,segID)

In [ ]:
# Iterates down the multiindex of motor neurons, selecting similar motor neuron segID to query for presynaptic partners.
# Inputs: client, mn_index and optional side
# returns: 

t1_mns_df = client.materialize.query_table('motor_neuron_table_v7',timestamp=connectome_create.get_timestamp())
t1_mns_df.index = connectome_create.mn_multi_index(t1_mns_df)
All = slice(None)
mn_index = t1_mns_df.loc[('L',All,All,All,All,All,All),:]
full_left_df = client.materialize.synapse_query(post_ids = t1_mns_df.loc['L',:].pt_root_id.to_list(),timestamp=connectome_create.get_timestamp())
print(full_left_df.shape)

In [ ]:
t1_mns_df.shape

In [ ]:
t1_mns_df.index.to_frame().reset_index(drop=True).to_csv('./dfs_saved/mns_to_make_public_20230524.csv')

In [ ]:
df = full_left_df.loc[:,['pre_pt_root_id','post_pt_root_id']]

# Then use factorize, which encodes each occurance in a list, in this case of paired root_ids, pre and post
ij,tups = pd.factorize(list(zip(*map(df.get,df))))

# Then create a dictionary out of each occurance, storing the number of factorized codes
result = dict(zip(tups, np.bincount(ij)))

# Finally, turn the result into series, with the tuples of pt_root_ids as the multiindex
tupseries = pd.Series(result)

thresh = 3
# Finally, threshold by the connection strength and perfom the unstack operation
thresholdedtupseries = tupseries[tupseries>=thresh]
prelim_pre_df = thresholdedtupseries.unstack(fill_value=0)

# takes ~ 15 seconds,
print(prelim_pre_df.shape)
pre_to_mn_df = prelim_pre_df.loc[:,mn_index.index.get_level_values('segID').to_list()]
pre_to_mn_df.columns = mn_index.index

In [ ]:
soma_table = client.materialize.query_table('neuron_somas_dec2022',timestamp=connectome_create.get_timestamp())
neckns_df = client.materialize.query_table('neck_connective',timestamp=connectome_create.get_timestamp())
all_sensory = connectome_create.get_unduplicated_sensory_axon_table(client)

pmn_index = pre_to_mn_df.index.to_frame()
pmn_index.rename(columns={0:'segID'},inplace=True)
pmn_index['motor'] = pmn_index.segID.isin(mn_index.pt_root_id)
pmn_index['has_soma'] = pmn_index.segID.isin(soma_table.pt_root_id)
pmn_index['sensory'] = pmn_index.segID.isin(all_sensory.pt_root_id)
pmn_index['neck'] = pmn_index.segID.isin(neckns_df.pt_root_id)
pmn_index['local'] = False
pmn_index.neck.sum(axis=0)
pre_to_mn_df.index = pd.MultiIndex.from_frame(pmn_index)

# Sort the matrix

In [ ]:
s = pre_to_mn_df.sum(axis=0)
pre_to_mn_df = pre_to_mn_df[s.sort_values(ascending=True).index]

new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
pre_to_mn_df = pre_to_mn_df[new_idx]

# Create cell_class

In [ ]:
### Motor neurons
# segID	motor	has_soma	sensory	 neck	local	
m_r_tup = (All,True,All,All,All,All)
mns = pre_to_mn_df.loc[m_r_tup,:]
mns = mns.loc[mns.sum(axis=1).sort_values(ascending=False).index,:]
print('mns shape: {}'.format(mns.shape))

### sensory neurons
# segID	motor	has_soma	sensory	 neck	local	
s_r_tup = (All,False,False,True,False,False)
sensory = pre_to_mn_df.loc[s_r_tup,:]
sensory = sensory.loc[sensory.sum(axis=1).sort_values(ascending=False).index,:]
print('sensory shape: {}'.format(sensory.shape))

### descending neurons
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True,All),:]
descending = descending.loc[descending.sum(axis=1).sort_values(ascending=False).index,:]
print('descending shape: {}'.format(descending.shape))

### How many ascending neurons # 138
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
ascending = ascending.loc[ascending.sum(axis=1).sort_values(ascending=False).index,:]
print('ascending shape: {}'.format(ascending.shape))

### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,False),:]
vncns = vncns.loc[vncns.sum(axis=1).sort_values(ascending=False).index,:]
print('vncns shape: {}'.format(vncns.shape))

## strange motor neurons
sensmns = pre_to_mn_df.loc[(All,False,True,True,False,False),:]
sensmns = sensmns.loc[sensmns.sum(axis=1).sort_values(ascending=False).index,:]
print('sensmns shape: {}'.format(sensmns.shape))

(mns.shape[0]+sensory.shape[0]+descending.shape[0] + ascending.shape[0] + vncns.shape[0] + sensmns.shape[0] )

# Look at fragments briefly

In [ ]:
# Neurons with a soma and not ascending
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:].copy()
### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
fragments = fragments.loc[fragments.sum(axis=1).sort_values(ascending=False).index,:]

total = fragments.sum(axis=1)
tot = total.cumsum()#/total.sum() # The fraction of total input from a given fragment
n = np.array([i for i in range(len(tot))])  # assign a number to neurons
frc = n/n.sum()

fig = plt.figure(1, figsize=(8, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

# ax1.scatter(x, y)
ax1.plot(n,tot)#, marker='o')
plt.sca(ax1)
plt.title('Total synapses from N fragments')
plt.ylabel('M synapses')
plt.xlabel('N fragments')
plt.show()
# locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

In [ ]:
connectome_create.save_pre_to_mn_df(pre_to_mn_df,ext='edited')

In [ ]:
ordered_pMN_to_MN_df = pd.concat([
    mns,
    descending,
    sensory,
    vncns,
    ascending,
    sensmns,
    fragments
])
print(ordered_pMN_to_MN_df.shape)

In [ ]:
connectome_create.save_pre_to_mn_df(ordered_pMN_to_MN_df,ext='ordered')

# Load Premotor connectivity matrix

In [ ]:
pre_to_mn_df = ordered_pMN_to_MN_df # connectome_create.load_pre_to_mn_df(ext='ordered')


In [ ]:
pre_to_mn_df

In [ ]:
All = slice(None)

# # How many motor neurons are presynaptic to motor neurons
# # segID	motor	has_soma	sensory	 neck	local	
motor = pre_to_mn_df.loc[(All,True,True,All,False,False),:]
# descending.index.get_level_values('segID').to_list()
print(motor.shape)

# How many sensory neurons # 94
# segID	motor	has_soma	sensory	 neck	local	
sensory = pre_to_mn_df.loc[(All,False,False,True,False,False),:]
# descending.index.get_level_values('segID').to_list()
print(sensory.shape)

# How many descending neurons # 217 214?
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True),:]
# descending.index.get_level_values('segID').to_list()
print(descending.shape)

# How many motor neurons are presynaptic to motor neurons
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
# descending.index.get_level_values('segID').to_list()
print(ascending.shape)
# some ascending that should be ascending
# 648518346507084872
# 648518346480786912
# 648518346490374748
sid = 648518346504729203
ascending.index.get_level_values('segID')==sid
# All are here

# Fragments?
# segID	motor	has_soma	sensory	 neck	local	
sensmns = pre_to_mn_df.loc[(All,False,True,True,False,False),:]
print(sensmns.shape)

# Fragments?
# segID	motor	has_soma	sensory	 neck	local	
fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
print(fragments.shape)

# Local and intersegmental? # 880
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,False),:]
# descending.index.get_level_values('segID').to_list()
print(vncns.shape)


# Find intersegmental neurons
Go through the vnc premotor neurons and look for a synapse outside the left hand box.
Once you find one, move to the next premotor neuron

In [ ]:
# Soma box
soma_table = client.materialize.query_table('somas_dec2022',timestamp=connectome_create.get_timestamp())
soma_table.index = soma_table.pt_root_id

In [ ]:
sx = [-3000, 3000]
sy = [-3000, 3000]
sz = [-300, 300]

# T1 bounding box, click on neuroglancer instance
All = slice(None)
x = [3000 , 38000]
y = [90483, 123190] # extended the boundaries a bit 5/10/22
z = [980, 3858]

t1 = lambda b: (b[0]>=x[0]) & (b[0]<=x[1]) & (b[1]>=y[0]) & (b[1]<=y[1]) & (b[2]>=z[0]) & (b[2]<=z[1])

def get_non_T1_inputs(iln_syn_df, s):
    in_T1_box = iln_syn_df.loc[:,['post_pt_position']].applymap(t1)
    pts_out_of_T1 = iln_syn_df.loc[~in_T1_box.post_pt_position,:]
    pts_in_T1 = iln_syn_df.loc[in_T1_box.post_pt_position,:]
    nearsoma = lambda b: (b[0]>=s[0]+sx[0]) & (b[0]<=s[0]+sx[1]) & (b[1]>=s[1]+sy[0]) & (b[1]<=s[1]+sy[1]) & (b[2]>=s[2]+sz[0]) & (b[2]<=s[2]+sz[1])
    in_soma_box = pts_out_of_T1.loc[:,['post_pt_position']].applymap(nearsoma)
    pts_out_of_T1 = pts_out_of_T1.loc[~in_soma_box.post_pt_position,:]
    return pts_out_of_T1

In [ ]:
from datetime import date
vncnsindex = vncns.index.to_frame()
vncnsindex.index = vncnsindex.index.get_level_values('segID')
vncnsindex['non_T1_count'] = 0

row_i = 0
row_f = 99
cnt = 0
while row_i<vncnsindex.shape[0]:
    print('Querying [{}:{}]'.format(row_i,row_f))
    rows_df = vncnsindex.iloc[row_i:row_f,:]
    iln_syn_df = client.materialize.synapse_query(post_ids = rows_df.index.get_level_values('segID').to_list(),timestamp=connectome_create.get_timestamp())
    for idx in rows_df.index.get_level_values('segID').to_list():
        try:
            s=soma_table.loc[idx,:].pt_position
        except KeyError:
            if 'fraglist' not in locals():
                today = date.today()
                d1 = today.strftime("%Y%m%d")
                fn = './dfs_saved/' + 'frag_df' + '_' + d1 + '.pkl'
                fraglist= pd.read_pickle(fn)
            if sid in fraglist.pt_root_id.to_list():
                print('{} is a known fragment of a local neuron'.format(sid))
                vncnsindex.loc[idx,'local'] = True
        
        non_T1_inputs = get_non_T1_inputs(iln_syn_df.loc[iln_syn_df['post_pt_root_id']==idx], s)
        if non_T1_inputs.empty:
            #print('{} is local'.format(sid))
            vncnsindex.loc[idx,'local'] = True
        else: 
            #print('{} has {} inputs outside of T1'.format(sid,non_T1_inputs.shape[0]))
            vncnsindex.loc[idx,'non_T1_count'] = non_T1_inputs.shape[0]
        
    print('Done [{}:{}]'.format(row_i,row_f))
    row_i = row_f+1
    row_f = min([row_i+99,vncnsindex.shape[0]])
    
vncns.index = pd.MultiIndex.from_frame(vncnsindex)


# Reconcatenate dataframes

In [ ]:
mnindex= motor.index.to_frame()
mnindex.index = mnindex.index.get_level_values('segID')
mnindex['non_T1_count'] = 0
motor.index = pd.MultiIndex.from_frame(mnindex)

descindex= descending.index.to_frame()
descindex.index = descindex.index.get_level_values('segID')
descindex['non_T1_count'] = 0
descending.index = pd.MultiIndex.from_frame(descindex)

sensoryindex= sensory.index.to_frame()
sensoryindex.index = sensoryindex.index.get_level_values('segID')
sensoryindex['non_T1_count'] = 0
sensory.index = pd.MultiIndex.from_frame(sensoryindex)

ascindex= ascending.index.to_frame()
ascindex.index = ascindex.index.get_level_values('segID')
ascindex['non_T1_count'] = 0
ascending.index = pd.MultiIndex.from_frame(ascindex)

sensmnsindex= sensmns.index.to_frame()
sensmnsindex.index = sensmnsindex.index.get_level_values('segID')
sensmnsindex['non_T1_count'] = 0
sensmns.index = pd.MultiIndex.from_frame(sensmnsindex)

frgindex= fragments.index.to_frame()
frgindex.index = frgindex.index.get_level_values('segID')
frgindex['non_T1_count'] = 0
fragments.index = pd.MultiIndex.from_frame(frgindex)

ordered_pMN_to_MN_df = pd.concat([
    motor,
    descending,
    sensory,
    vncns,
    ascending,
    sensmns,
    fragments,
])

In [ ]:
connectome_create.save_pre_to_mn_df(ordered_pMN_to_MN_df,ext='vncns_inter_vs_local')

# Reorder

In [ ]:
# segID	motor	has_soma	sensory	 neck	local	
pre_to_mn_df = ordered_pMN_to_MN_df

m_r_tup = (All,True,All,All,All,All)
motor = pre_to_mn_df.loc[m_r_tup,:]
motor = motor.loc[motor.sum(axis=1).sort_values(ascending=False).index,:]
print(motor.shape)

### sensory neurons
# segID	motor	has_soma	sensory	 neck	local	
s_r_tup = (All,False,False,True,False,False)
sensory = pre_to_mn_df.loc[s_r_tup,:]
sensory = sensory.loc[sensory.sum(axis=1).sort_values(ascending=False).index,:]
print(sensory.shape)

### weird mn neurons
# segID	motor	has_soma	sensory	 neck	local	
w_r_tup = (All,False,True,True,False,False)
sensmns = pre_to_mn_df.loc[w_r_tup,:]
sensmns = sensmns.loc[sensmns.sum(axis=1).sort_values(ascending=False).index,:]
print(sensmns.shape)

### descending neurons
# segID	motor	has_soma	sensory	 neck	local	
descending = pre_to_mn_df.loc[(All,False,False,False,True,All),:]
descending = descending.loc[descending.sum(axis=1).sort_values(ascending=False).index,:]
print(descending.shape)

### How many ascending neurons # 138
# segID	motor	has_soma	sensory	 neck	local	
ascending = pre_to_mn_df.loc[(All,False,True,False,True,False),:]
ascending = ascending.loc[ascending.sum(axis=1).sort_values(ascending=False).index,:]
print(ascending.shape)

### VNC neurons
# segID	motor	has_soma	sensory	 neck	local	
vncns = pre_to_mn_df.loc[(All,False,True,False,False,All,All),:]
vncns = vncns.loc[vncns.sum(axis=1).sort_values(ascending=False).index,:]
print(vncns.shape)

fragments = pre_to_mn_df.loc[(All,False,False,False,False,False),:]
fragments.shape

# Reclassify
Should some of these be local neurons instead?

In [ ]:

local = vncns.loc[(All,False,True,False,False,True,All),:]
print('local shape:{}'.format(local.shape))

# peel off some local neurons from the intersegmental neurons
intrseg = vncns.loc[(All,False,True,False,False,False,All),:]
print('inter shape:{}'.format(intrseg.shape))

index = intrseg.index.to_frame()
index.index = index.index.get_level_values('segID')
somenonT1 = index.non_T1_count<20
intrseg.loc[somenonT1.to_list(),:]

possible_local = intrseg.loc[somenonT1.to_list(),:]
print(possible_local.shape)

# Order local_p by numbers of synapses
local_p = pd.concat([local,possible_local])
local_p = local_p.loc[local_p.sum(axis=1).sort_values(ascending=False).index,:]

intrseg_m = intrseg.loc[~somenonT1.to_numpy(),:]

print(intrseg_m.shape)

# Print the next tranche of neurons to see if any others should be typed and local
mi,arr = intrseg_m.index.sortlevel(level='non_T1_count')
mi_df = mi.to_frame()

# Now just look for that giant 19B neuron and move it to the local stack
pmns = pd.DataFrame()
pmns['pt_position'] = [[52132, 120825, 1557]]

# looking to add this one particular neuron:
cv = cloudvolume.CloudVolume(client.info.get_datastack_info()['segmentation_source'], use_https=True)
segID = utils.segIDs_from_pts_service(pmns.pt_position,cv)

if any(pre_to_mn_df.index.get_level_values('segID').isin([segID[0]])):
    print('Moving large neuron {} from intersegmental to local group'.format(segID))
    local_p = pd.concat([local_p,intrseg_m.loc[segID]])
    local_p = local_p.loc[local_p.sum(axis=1).sort_values(ascending=False).index,:]


locidx = local_p.index.to_frame()
# locidx.index = locidx.index.get_level_values('segID').to_list()
locidx.local = True
locmi = pd.MultiIndex.from_frame(locidx)
local_p.index = locmi
print(local_p.shape)

intrseg_m = intrseg_m.loc[mi_df.segID!=segID[0],:]
print('local shape:{} ({})'.format(local_p.shape,locidx.local.sum()))
print('inter shape:{}'.format(intrseg_m.shape))

In [ ]:
local_p.head()
segID[0]
local_p.loc[648518346480658945]

## Sort cell classes by similarity onto mns

### Descending neurons

In [ ]:
#print(descending.shape)
sim_mat = cosine_similarity(descending.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
descending_clustered = descending.iloc[clustered_order,:].copy()
descending_clustered.shape

### sensory neurons

In [ ]:
sim_mat = cosine_similarity(sensory.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
sensory_clustered = sensory.iloc[clustered_order,:].copy()
sensory_clustered.shape

### local neurons

In [ ]:
sim_mat = cosine_similarity(local_p.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
local_clustered = local_p.iloc[clustered_order,:].copy()
local_clustered.shape

### intersegmental neurons

In [ ]:
sim_mat = cosine_similarity(intrseg_m.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
intrseg_clustered = intrseg_m.iloc[clustered_order,:].copy()
intrseg_clustered.shape

In [ ]:
sim_mat = cosine_similarity(ascending.to_numpy())

# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None).fit(sim_mat)

# plot the top three levels of the dendrogram
clustered_order = utils.plot_dendrogram(model, truncate_mode="level", p=6000) # p truncate mode
clustered_order = np.array(clustered_order).astype(int) # convert strins into integers
# print(clustered_order)
ascending_clustered = ascending.iloc[clustered_order,:].copy()
ascending_clustered.shape

In [ ]:
clustered_pMN_to_MN_df = pd.concat([
    motor,
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_p, # local_clustered,
    ascending_clustered,
    sensmns,
    fragments
])
print(clustered_pMN_to_MN_df.shape)

clustered_pMN_to_MN_df_no_frags = pd.concat([
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_p, # local_clustered,
    ascending_clustered,
])
print(clustered_pMN_to_MN_df_no_frags.shape)

clustered_pMN_to_MN_df_with_clusteredvnc = pd.concat([
    descending_clustered,
    sensory_clustered,
    intrseg_clustered,
    local_clustered,
    ascending_clustered,
])
print(clustered_pMN_to_MN_df_with_clusteredvnc.shape)

# ordered_pMN_to_MN_df_no_desc = pd.concat([
#     sensory,
#     vncns,
#     ascending
# ])
# print(ordered_pMN_to_MN_no_desc_df.shape)


# ordered_pMN_to_MN_df_all = pd.concat([
#     descending,
#     sensory,
#     vncns,
#     ascending,
#     fragments
# ])
# print(ordered_pMN_to_MN_df_all.shape)


# Save the clustered_pMN_to_MN_df

In [ ]:
connectome_create.save_pre_to_mn_df(clustered_pMN_to_MN_df,ext='clustered_ordered')

# Load Motor neuron table and determine cell classes 
My current hierarchy would be: Region (brain,vnc,descending,ascending), segment (T1,2,3,Ab), class (motor, sensory, intersegmental, local), neurotransmitter, primary neurite bundle/hemilineage, type

In [ ]:
all_pre_to_mn_df = clustered_pMN_to_MN_df # connectome_create.load_pre_to_mn_df(ext='clustered_ordered')

all_pre_to_mn_df['cell_class']  = ''
print(all_pre_to_mn_df.shape)

All = slice(None)
                    # segID	motor	has_sma	sensory	neck	local   T1_count
all_pre_to_mn_df.loc[(All,  True,   True,   False,  False,  False,  All),'cell_class'] = 'motor'
all_pre_to_mn_df.loc[(All,  True,   True,   True,  False,  False,  All),'cell_class'] = 'motor'
all_pre_to_mn_df.loc[(All,  False,  False,    True,   False,  False,  All),'cell_class'] = 'sensory' 
all_pre_to_mn_df.loc[(All,  False,  True,    True,   False,  False,  All),'cell_class'] = 'sensemns' # always has those weird neurons in there
all_pre_to_mn_df.loc[(All,  False,  False,  False,  True,   False,  All),'cell_class'] = 'descending'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  True,   False,  All),'cell_class'] = 'ascending'
all_pre_to_mn_df.loc[(All,  False,  False,  False,  False,  False,  All),'cell_class'] = 'fragment'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  False,  True,   All),'cell_class'] = 'local'
all_pre_to_mn_df.loc[(All,  False,  True,   False,  False,  False,  All),'cell_class'] = 'intersegmental'
print(all_pre_to_mn_df.cell_class.value_counts())

all_pre_to_mn_df = all_pre_to_mn_df.set_index(keys=['cell_class'],append=True,drop=True,verify_integrity=True)
all_pre_to_mn_df = all_pre_to_mn_df.reset_index(level=['motor','has_soma','sensory','neck','local','non_T1_count'],drop=True)
all_pre_to_mn_df = all_pre_to_mn_df.reorder_levels(['cell_class','segID'],'index')
print(all_pre_to_mn_df.shape)

pre_to_mn_df = all_pre_to_mn_df.loc[(['descending','sensory','intersegmental','local','ascending']),:]
print(pre_to_mn_df.shape)

pre_to_mn_df.head()


# Note 8/2/22 - the the sensory group is currently 117. This includes 111 actual sensory neurons, likely after some proofreading. 
# Then there are 6 sensory neurons with somas. This should change on 8/3, there are still the bilateral mns, but some of the sensory snarls have been proofread


In [ ]:
connectome_create.save_pre_to_mn_df(all_pre_to_mn_df,ext='pre_match_to_pool_w_fragments')

# Assigning preMNs to motor modules
This script answers the following question: Is there a stable match where we can assign a local premotor neuron to a muscle group?
The idea comes from the med school matching algorithm and aims to produce an order to the premotor neurons where the "most important" neurons for a given muscle is assigned to that muscle. This will allow us to look at the connectivity of the local premotors and say, e.g., "this important neuron for tibia flexors is also connected to femur flexors". So, what does "important" mean? To assign premotors to motor modules, for each premotor neuron, we rank the motor modules by the number of combined inputs to the motor modules. For each motor modules, we rank premotor neurons by their contribution to the cluster unit vector. Then we let the matching algorithm work on those rankings. This ranking system is premotor/student optimal, and the lists for the motor modules/school are not limited, so it should aggregate premotor neurons just to their prefered module.

In [ ]:
from match.RothPeranson import MatchController
plot_logical = False
show_logical = False

new_pre_to_mn = None
sho_pre_to_mn = None

pool_capacity = 65  # This limits how many neurons make up the main tibia flexor pool
pool_capacity = 200 # This allows nearly as many premotor neurons into any pool that want to

mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]

# quick sanity check, all the mns assigned to a pool
# total=0
# for key in pool_keys:
#     a = pre_to_mn_df.loc[:,mpool_dict[key]].shape[1]
#     total = total+a
#     print('{}: {}'.format(key,a))
# print('{} total'.format(total))

for cclass_tup in [('descending',All,), ('sensory',All,), ('ascending',All,),('intersegmental',All,),('local',All,)]:
# for cclass_tup in [('local',All,)]:

    cell_class = pre_to_mn_df.loc[cclass_tup,:]

    figfname = './figpanels/{}_match_matrix.svg'.format(cclass_tup[0])
    ylabelstring = '{} neurons'.format(cclass_tup[0])
    print('Starting {}, {} premotor neurons'.format(ylabelstring,cell_class.shape[0]))

    # Make a data frame of inputs to a pool
    pre_to_pool_df = None
    for key in pool_keys:
        if pre_to_pool_df is None:
            pre_to_pool_df = cell_class.loc[:,mpool_dict[key]].sum(axis=1)
        else:    
            pre_to_pool_df = pd.concat([pre_to_pool_df,cell_class.loc[:,mpool_dict[key]].sum(axis=1)],axis=1)
    pre_to_pool_df.columns = pool_keys
    pre_to_pool_df

    # Rank the motor pools for each premotor neuron
    pre_to_pool_ranks_df = pre_to_pool_df.assign(**pre_to_pool_df.iloc[:, 0:].rank(axis = 1, ascending = False).astype(int))
    pre_to_pool_ranks_df.index = pre_to_pool_ranks_df.index.get_level_values('segID')

    # take the transpose, so each premotor neuron is a column and fill in the values with the ranked pool
    premn_prefs_for_pools = pre_to_pool_ranks_df.T
    pools = np.array(premn_prefs_for_pools.index.to_list())
    for idx in premn_prefs_for_pools:
            premn_prefs_for_pools[idx] = pools[(premn_prefs_for_pools[idx].to_numpy(dtype=int)- 1).argsort()]
    premn_prefs_for_pools = premn_prefs_for_pools.reset_index(drop=True)
    premn_prefs_for_pools.columns = premn_prefs_for_pools.columns.astype(str)

    # Now create a DF where the motor pools rank the premotor neurons. 
    # Do this by ranking along the pool

    pool_ranks_of_pre_df = pre_to_pool_df.assign(**pre_to_pool_df.iloc[0:,:].rank(axis = 0, ascending = False).astype(int))
    pool_ranks_of_pre_df.index = pool_ranks_of_pre_df.index.get_level_values('segID')

    # rank the premotors
    pool_prefs_for_premns = pool_ranks_of_pre_df
    premns = np.array(pool_prefs_for_premns.index.astype(str).to_list())
    for idx in pool_prefs_for_premns:
            pool_prefs_for_premns[idx] = premns[(pool_prefs_for_premns[idx].to_numpy(dtype=int)- 1).argsort()]
    pool_prefs_for_premns = pool_prefs_for_premns.reset_index(drop=True)

    pool_capacities = pool_prefs_for_premns.count()-(pool_prefs_for_premns.shape[0]-pool_capacity)
    pool_capacities_df = pool_capacities.to_frame()

    pool_capacities_df = pool_capacities_df.rename(columns = {0: 'places'})

    ## Now run the match algorithm
    match = MatchController(pool_prefs_for_premns, premn_prefs_for_pools,pool_capacities_df)
    match.start_match()
    match_dict = match.results_dict()

    # Now reorder according to the match
    lidx = cell_class.index.to_frame()
    lidx['preferred_pool'] = ''
    for idx,row in lidx.iterrows():
        lidx.loc[idx,'preferred_pool'] = match_dict[str(row.segID)]
    lidx
        
    cell_class.index = pd.MultiIndex.from_frame(lidx)
    cell_class = cell_class.reorder_levels(['cell_class','preferred_pool','segID'],'index')

    s = cell_class.sum(axis=1)
    cell_class = cell_class.loc[s.sort_values(ascending=False).index,:]
    cell_class = cell_class.loc[(All,pool_keys,All),:]

    cell_class.head()

    %config InlineBackend.figure_formats = ['png']
    %matplotlib inline

    if plot_logical:
        cmap = utils.white_dense()

        mn_mi = cell_class.columns.to_frame()
        lbls = utils.mn_labels(mn_mi,depth='rank')
        fig = plt.figure(1, figsize = [15,25])
        ax = sns.heatmap(np.log10(cell_class.to_numpy()+1), xticklabels=lbls, cmap=cmap)
        cbar = ax.collections[0].colorbar
        # here set the labelsize by 20
        # cbar.ax.tick_params(labelsize=20)
        cbar.set_label(label = 'log 10 # of synapses', size=24)
        plt.xlabel('Motor Neurons', fontsize =18)
        plt.ylabel(ylabelstring, fontsize =18)
        plt.yticks(fontsize = 16)
        plt.xticks(fontsize = 16)
        plt.show()

        fig.savefig(figfname,format='svg')

    if show_logical:
        # put in a gap, just for show
        cell_class_for_show = None
        pool_gap = cell_class.iloc[:,0].copy().to_frame()
        for col in pool_gap.columns:
            pool_gap[col].values[:] = 2

        cell_class_for_show = None
        for key in pool_keys:
            if cell_class_for_show is None:
                cell_class_for_show = cell_class.loc[:,mpool_dict[key]].copy()
            else:    
                cell_class_for_show = pd.concat([cell_class_for_show,pool_gap.copy()],axis=1)
                cell_class_for_show = pd.concat([cell_class_for_show,cell_class.loc[:,mpool_dict[key]]],axis=1)
    
        class_gap = cell_class_for_show.iloc[0:10,:].copy()
        for col in class_gap.columns:
            class_gap[col].values[:] = 2
        if sho_pre_to_mn is None:
            sho_pre_to_mn = cell_class_for_show.copy()
        else:
            sho_pre_to_mn = pd.concat([sho_pre_to_mn,class_gap.copy()],axis=0)
            sho_pre_to_mn = pd.concat([sho_pre_to_mn,cell_class_for_show],axis=0)

    if new_pre_to_mn is None:
        new_pre_to_mn = cell_class
    else:
        new_pre_to_mn = pd.concat([new_pre_to_mn,cell_class],axis=0)


if show_logical:
    fig = plt.figure(1, figsize = [15,25])
    ax = sns.heatmap(np.log10(sho_pre_to_mn.to_numpy()+1), xticklabels=lbls, cmap=cmap)
    cbar = ax.collections[0].colorbar
    # here set the labelsize by 20
    # cbar.ax.tick_params(labelsize=20)
    cbar.set_label(label = 'log 10 # of synapses', size=24)
    plt.xlabel('Motor Neurons', fontsize =18)
    plt.ylabel(ylabelstring, fontsize =18)
    plt.yticks(fontsize = 16)
    plt.xticks(fontsize = 16)
    plt.show()

    fig.savefig('matched_premotor_to_motor',format='svg')

In [ ]:
connectome_create.save_pre_to_mn_df(new_pre_to_mn,ext='matched_to_pool')

# Add local neuron classification and hemilineage to matrix

In [ ]:
pre_to_mn_df = new_pre_to_mn

In [ ]:
pmn_annotation_table = client.materialize.query_table('t1l_local_premotor_table_v3',timestamp=connectome_create.get_timestamp())


# pmn_annotation_table.classification_system.value_counts()
# pmn_annotation_table.cell_type.value_counts()

In [ ]:
# utils.save_df_csv(pmn_annotation_table,name='t1l_local_premotor_table_v3')

In [ ]:
# how many of the pmn seg is are not in the matrix
pmn_index = pre_to_mn_df.index.to_frame()
pmn_index = pmn_index.set_index(keys=['segID'],drop=False)
local_pmns  = pmn_index.loc[pmn_index.cell_class=='local']


local_pmns_to_type = local_pmns.loc[~local_pmns.segID.isin(pmn_annotation_table.pt_root_id)]
utils.save_df_csv(local_pmns_to_type,name='t1l_local_premns_to_type')
local_pmns_to_type.segID.to_list()

In [ ]:
# just add classifications and hemilineage to the index, then put the index back on and reorder
pmn_index = pre_to_mn_df.index.to_frame()
pmn_index = pmn_index.set_index(keys=['segID'],drop=False)

pmn_index['classification_system'] = None
pmn_index['cell_type'] = None

for clsys in pmn_annotation_table.classification_system.unique():
    pmn_index.loc[pmn_index.segID.isin(pmn_annotation_table.loc[pmn_annotation_table.classification_system==clsys,'pt_root_id']),'classification_system'] = clsys
    
for ctype in pmn_annotation_table.cell_type.unique():
    pmn_index.loc[pmn_index.segID.isin(pmn_annotation_table.loc[pmn_annotation_table.cell_type==ctype,'pt_root_id']),'cell_type'] = ctype

pre_to_mn_df.index = pd.MultiIndex.from_frame(pmn_index)
pre_to_mn_df = pre_to_mn_df.reorder_levels(['cell_class','preferred_pool','classification_system','cell_type','segID'],'index')


# Add neurotransmitters

## Create lookuptable

In [ ]:
cell_types = pre_to_mn_df.index.get_level_values('cell_type').unique().to_numpy()
nt_table = pd.read_csv('./annotations_hl/LacinHLTable.csv')
import re
pattern = re.compile("R[0-9]")

hl_lut = {'cell_type':cell_types,'hemilineage':cell_types}
hl_lut = pd.DataFrame(data=hl_lut)

for idx,r in hl_lut.iterrows():
    try:
        if np.isnan(r.hemilineage):
            continue
    except TypeError:
        pass
    hl = r.hemilineage

    if hl[0:2]=='RD':
        hl = '24B'
        # print('string was RD, now {}'.format(hl))

    if pattern.match(hl) or hl=='RVD' or hl=='Rcore_':
        # print('string was {}, now 8A'.format(hl))
        hl = '8A'

    if not hl.isalnum():
        hl = hl[0:-1]
        # print(hl)
    
    if not hl.isalnum():
        hl=hl[0:hl.find('_')]
        # print(hl)

    if hl == '9Ac':
        hl='9A'
        # print(hl)

    hl_lut.loc[idx,'hemilineage'] = hl

# merge with neurotransmitter table to get a lookuptable
hl_lut = hl_lut.merge(nt_table,how='outer',left_on='hemilineage',right_on='HL')
hl_lut = hl_lut.loc[~hl_lut.hemilineage.isna(),:]

hl_lut_reduced = hl_lut[['cell_type','NT']]

In [ ]:
# just operate on the index, where the nt will be
pmn_index_df = pre_to_mn_df.index.to_frame().reset_index(drop=True)

# join the index on cell_type, with NT
pmn_index_df = pmn_index_df.join(hl_lut_reduced.set_index('cell_type'),how='left',on='cell_type')

# reorder the matrix
pmn_index_df = pmn_index_df[['cell_class','preferred_pool','NT','classification_system','cell_type','segID']]
pmn_index_df.NT.value_counts()

# Classify sensory neurons

In [ ]:
all_sensory = connectome_create.get_unduplicated_sensory_axon_table(client)

all_sensory = all_sensory[['pt_root_id','classification_system','cell_type']]
all_sensory = all_sensory.rename({'pt_root_id':'segID'},axis=1)

# join the index on cell_type, with NT
pmn_index_df = pmn_index_df.merge(all_sensory,how='left',on='segID',suffixes=['_x','_y'])

sens_rows = (pmn_index_df.cell_class == 'sensory') & (pmn_index_df.classification_system_x.isna())
pmn_index_df.loc[sens_rows,'classification_system_x'] = pmn_index_df.loc[sens_rows].classification_system_y
pmn_index_df.loc[sens_rows,'cell_type_x'] = pmn_index_df.loc[sens_rows].cell_type_y

pmn_index_df.loc[pmn_index_df.cell_class == 'sensory','NT'] = 'Ach'
pmn_index_df = pmn_index_df.drop(['classification_system_y','cell_type_y'],axis=1)
pmn_index_df = pmn_index_df.rename(columns={'classification_system_x':'classification_system','cell_type_x':'cell_type'})

pre_to_mn_df.index = pd.MultiIndex.from_frame(pmn_index_df)

glia = pre_to_mn_df[pre_to_mn_df.index.get_level_values('cell_type') == 'Gial_error']
no_glia = pre_to_mn_df[pre_to_mn_df.index.get_level_values('cell_type') != 'Gial_error']

In [ ]:
connectome_create.save_pre_to_mn_df(no_glia,ext='matched_typed_with_nt')

In [ ]:
print(no_glia.index.get_level_values('cell_class').value_counts())

In [ ]:
print(no_glia.index.get_level_values('preferred_pool').value_counts())

In [ ]:
print(no_glia.index.get_level_values('NT').value_counts())

In [ ]:
print(no_glia.index.get_level_values('classification_system').value_counts())

In [ ]:
print(no_glia.index.get_level_values('cell_type').value_counts())

In [ ]:
no_glia.head()